In [2]:
import findspark
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/18 07:59:58 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.131 instead (on interface wlp0s20f3)
23/07/18 07:59:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/18 07:59:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

#### Almacenamiento en caché

El almacenamiento en caché permite que Spark conserve los datos en todos los cálculos y operaciones.\
Es una de las técnicas más importantes de spark para acelerar los cálculos, especialmente cuando se trata de cálculos interactivos.\
Funciona almacenando el RDD tanto como sea posible en la memoria. Si los datos que se solicitan para almacenar en caché son más grandes que la memoria disponible, el rendimiento disminuirá porque se utilizará disco en lugar de memoria.
